In [34]:
import pandas as pd
import csv


In [35]:
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('cpv/all.csv', sep=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, usecols=[1, 2, 3])
df.head()

,desc,lang,cpv
0,Marché de travaux d'entretien en plomberie des parties communes et des logements habites des cites de 13 habitat.,FR,45330000
1,Mantenimiento base de datos de instalaciones (BDI) y Scada.,ES,50800000
2,Aménagement des consultations externes au rez-de-chaussée du bâtiment principal.,FR,45000000
3,"Limpieza, carboneo y mantenimiento centrales térmicas y ciclos combinados.",ES,76100000
4,"— la location, l'installation et la desinstallation de système embarqué de géolocalisation dans les engins d'exploitation routière,",FR,79714100


In [36]:
df.dropna(inplace=True)

def int_to_hierarchy(int_cpv):
    v = str(int(int_cpv))
    return [subcategory for subcategory in [v[:2], *v[2:]] if subcategory != '0']

df['doc_label'] = df.cpv.apply(int_to_hierarchy)

df.desc = df.desc.str.replace(r'\d{8}(-\d)?', '', regex=True)
df.head()

,desc,lang,cpv,doc_label
0,Marché de travaux d'entretien en plomberie des parties communes et des logements habites des cites de 13 habitat.,FR,45330000,"[45, 3, 3]"
1,Mantenimiento base de datos de instalaciones (BDI) y Scada.,ES,50800000,"[50, 8]"
2,Aménagement des consultations externes au rez-de-chaussée du bâtiment principal.,FR,45000000,[45]
3,"Limpieza, carboneo y mantenimiento centrales térmicas y ciclos combinados.",ES,76100000,"[76, 1]"
4,"— la location, l'installation et la desinstallation de système embarqué de géolocalisation dans les engins d'exploitation routière,",FR,79714100,"[79, 7, 1, 4, 1]"


In [37]:
from transformers import AutoTokenizer

In [38]:
tokenizer = AutoTokenizer.from_pretrained("EuropeanParliament/EUBERT")


In [39]:
df['doc_token'] = df.desc[:100].apply(lambda x: tokenizer.tokenize(x))

In [40]:
df.head()

,desc,lang,cpv,doc_label,doc_token
0,Marché de travaux d'entretien en plomberie des parties communes et des logements habites des cites de 13 habitat.,FR,45330000,"[45, 3, 3]","[marche, de, travaux, d, ', entre, ##tien, en, plo, ##mb, ##erie, des, parties, communes, et, des, log, ##ements, hab, ##ites, des, cites, de, 13, habitat, .]"
1,Mantenimiento base de datos de instalaciones (BDI) y Scada.,ES,50800000,"[50, 8]","[mantenimiento, base, de, datos, de, instalaciones, (, bd, ##i, ), y, scad, ##a, .]"
2,Aménagement des consultations externes au rez-de-chaussée du bâtiment principal.,FR,45000000,[45],"[amen, ##agement, des, consultations, externe, ##s, au, rez, -, de, -, chau, ##sse, ##e, du, bati, ##ment, principal, .]"
3,"Limpieza, carboneo y mantenimiento centrales térmicas y ciclos combinados.",ES,76100000,"[76, 1]","[lim, ##pie, ##za, ,, carbone, ##o, y, mantenimiento, centrales, term, ##icas, y, ciclo, ##s, combina, ##dos, .]"
4,"— la location, l'installation et la desinstallation de système embarqué de géolocalisation dans les engins d'exploitation routière,",FR,79714100,"[79, 7, 1, 4, 1]","[—, la, location, ,, l, ', installation, et, la, desi, ##ns, ##tal, ##lation, de, systeme, embar, ##que, de, geo, ##loc, ##alisation, dans, les, eng, ##ins, d, ', exploitation, rou, ##tiere, ,]"


In [45]:
df[['doc_label', 'doc_token']][:100].to_json('cpv/input.json', orient='records')